In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Define the Model:
# We'll create a simple feedforward neural network for binary classification.

In [2]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x


# Prepare the Data:
#Let's assume you have your binary vectors and labels ready as X and y.

In [4]:
import numpy as np
K = 10
# Example data (replace with your actual data)
X = np.random.randint(0, 2, (1000, K)).astype(np.float32)
y = np.random.randint(0, 2, (1000, 1)).astype(np.float32)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

# Create a dataset and data loader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Train the Model:

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BinaryClassifier(input_size=K).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


Epoch [1/20], Loss: 0.6959
Epoch [2/20], Loss: 0.6924
Epoch [3/20], Loss: 0.6911
Epoch [4/20], Loss: 0.6902
Epoch [5/20], Loss: 0.6884
Epoch [6/20], Loss: 0.6881
Epoch [7/20], Loss: 0.6865
Epoch [8/20], Loss: 0.6843
Epoch [9/20], Loss: 0.6814
Epoch [10/20], Loss: 0.6827
Epoch [11/20], Loss: 0.6789
Epoch [12/20], Loss: 0.6737
Epoch [13/20], Loss: 0.6694
Epoch [14/20], Loss: 0.6658
Epoch [15/20], Loss: 0.6613
Epoch [16/20], Loss: 0.6565
Epoch [17/20], Loss: 0.6503
Epoch [18/20], Loss: 0.6503
Epoch [19/20], Loss: 0.6424
Epoch [20/20], Loss: 0.6366


In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 51.00%
